In [5]:
from dotenv import load_dotenv
load_dotenv()
from google.cloud.firestore import Client as FirestoreClient
from chat_history_service import ChatHistoryService
from llm_request_service import LLMRequestService
from typing import List, Dict, Any

## Define Functions

In [6]:
def get_chat_history_from_firestore(session_id: str) -> list[dict]:
    """
    Fetch chat history from Firestore for a given session ID.
    
    Args:
        session_id (str): The session ID to fetch chat history for.
    
    Returns:
        list[dict]: List of chat messages in the session.
    """
    chat_history_service = ChatHistoryService()
    chat_history = chat_history_service.get_chat_history_by_id(session_id)
    chat_history = sorted(chat_history, key=lambda x: x.get("create_time", ""))
    print(chat_history)
    return chat_history

In [7]:
def get_specific_data_from_history(history: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Retrieves specific data from the entire conversation history, maintaining the conversation flow.
    
    Args:
        history (List[Dict[str, Any]]): List of dictionaries containing message history.
    s
    Returns:
        List[Dict[str, Any]]: List of dictionaries, each containing user message, bot responses, and button label for each turn in the conversation.
    """
    # Sort the history by create_time
    sorted_history = sorted(history, key=lambda x: x.get("create_time"))
    
    # Initialize list to store the conversation turns
    conversation = []
    
    # Iterate through the history
    for message in sorted_history:
        turn = {}
        
        # Get user message
        user_msg = message.get("user_msg")
        if user_msg:
            turn["user_msg"] = user_msg
        
        # Get bot responses
        bot_response = message.get("bot_response", [])
        if isinstance(bot_response, list):
            bot_messages = []
            for response in bot_response:
                if isinstance(response, dict) and "message" in response:
                    bot_messages.append({"message": response["message"]})
            if bot_messages:
                turn["bot_response"] = bot_messages
        
        # Get button label
        button_label = message.get("button_label")
        if button_label:
            turn["button_label"] = button_label
        
        # Add the turn to the conversation if it's not empty
        if turn:
            conversation.append(turn)
    
    return conversation

In [8]:
def get_specific_chat_data(chat_history: str):
    """
    Fetches chat history from Firestore and selects specific data.

    Args:
        session_id (str): The session ID to fetch chat history for.

    Returns:
        Dict[str, Any]: Selected specific data from the chat history.
    """
    specific_chat_history = get_specific_data_from_history(chat_history)
    return specific_chat_history

In [9]:
def call_llm_service_summarize(conversation: str, prompt: str = "") -> str:
    """
    Call the LLM service to summarize the chat history.
    Args:
        conversation (str): The chat history to summarize.
        prompt (str): The prompt to send to the LLM service.
            If not provided, the default prompt will be fetched from Firestore.
    Returns:
        str: The summary of the chat history.
    """
    llm_request_service = LLMRequestService()
    chat_history_summary = llm_request_service.summarize_conversation(
        conversation, prompt
    )
    return chat_history_summary

In [10]:
def format_summary(summary: str) -> str:
    # Split the summary into sentences
    sentences = summary.split('. ')
    # Join the sentences with line breaks
    formatted_summary = '.\n'.join(sentences)
    return f"Summary:\n{formatted_summary}"

def count_output(text):
    """
    Count characters, words, and lines in the given text.
    """
    char_count = len(text)
    word_count = len(text.split())
    return char_count, word_count

In [11]:
def call_llm_service_queue(summary: str, customer_comment: str = "", service: str = "", telesales: str = "", techniek: str = "", activatie: str = "", tag_prompt: str = "") -> str:
    """
    Call the LLM service to tag the call queue.
    Args:
        summary (str): The ai generated summary of the conversation.
        tag_prompt (str): The prompt to send to the LLM service for tagging.
    Returns:
        str: Tagged call queue.
    """
    llm_request_service = LLMRequestService()
    tag_queue = llm_request_service.tag_callback(summary, customer_comment, service, telesales, techniek, activatie, tag_prompt)
    return tag_queue

## Gather Conversation

### Get conversation from session_id in Development

In [12]:
session_id = "20e62f9c-be3e-4d6e-978b-7d67dd5f10e8" # Replace with your actual session ID
chat_history = get_chat_history_from_firestore(session_id)
chat_history

[{'button_label': '', 'response_time': DatetimeWithNanoseconds(2025, 7, 24, 8, 24, 35, 205532, tzinfo=datetime.timezone.utc), 'intent_id': 'MT_SYSTEM_PRODUCTSELECT', 'sender': 'bot', 'bot_response': [{'attachment': None, 'subscription_brand': None, 'numbers_en': None, 'welcome_short': None, 'type': 'text', 'image': None, 'buttons': None, 'platform': None, 'user_type': None, 'message': 'Goedemorgen, ik ben chatbot Izzi van Odido.', 'fallback_length': None, 'link': None, 'html_en': None, 'product': None, 'billing_account': None, 'escapeOption': None, 'msisdnflag': None, 'html': None, 'linkList': None, 'numbers': None, 'randomizer': None, 'video': None, 'failed_event_trigger': None, 'message_en': None, 'bullets_en': None, 'bullets': None, 'language': None}, {'attachment': None, 'subscription_brand': None, 'numbers_en': None, 'welcome_short': None, 'type': 'button-list', 'image': None, 'buttons': [{'event': 'MT_SYSTEM_PRODUCTSELECT-choice;mobiel', 'label_en': 'Mobile', 'label': 'Mobiel'}, 

[{'button_label': '',
  'response_time': DatetimeWithNanoseconds(2025, 7, 24, 8, 24, 35, 205532, tzinfo=datetime.timezone.utc),
  'intent_id': 'MT_SYSTEM_PRODUCTSELECT',
  'sender': 'bot',
  'bot_response': [{'attachment': None,
    'subscription_brand': None,
    'numbers_en': None,
    'welcome_short': None,
    'type': 'text',
    'image': None,
    'buttons': None,
    'platform': None,
    'user_type': None,
    'message': 'Goedemorgen, ik ben chatbot Izzi van Odido.',
    'fallback_length': None,
    'link': None,
    'html_en': None,
    'product': None,
    'billing_account': None,
    'escapeOption': None,
    'msisdnflag': None,
    'html': None,
    'linkList': None,
    'numbers': None,
    'randomizer': None,
    'video': None,
    'failed_event_trigger': None,
    'message_en': None,
    'bullets_en': None,
    'bullets': None,
    'language': None},
   {'attachment': None,
    'subscription_brand': None,
    'numbers_en': None,
    'welcome_short': None,
    'type': 'but

In [13]:
conversation = get_specific_chat_data(chat_history)
conversation

[{'bot_response': [{'message': 'Goedemorgen, ik ben chatbot Izzi van Odido.'},
   {'message': 'Over welk product wil je iets vragen?'}]},
 {'bot_response': [{'message': 'Komen we er samen niet uit, dan zijn onze adviseurs er voor je.'},
   {'message': 'Wat kan ik voor je doen?'}],
  'button_label': 'Mobiel'},
 {'user_msg': 'videoland',
  'bot_response': [{'message': 'Met Videoland kijk je de beste series uit binnen- en buitenland. Het aanbod is erg ruim. Zo zit er voor iedereen wat tussen.'},
   {'message': 'Kijk Videoland waar en wanneer je maar wil. Thuis op de bank, of onderweg in de trein. Erg handig.'},
   {'message': 'Waarmee kan ik je helpen?'}]},
 {'bot_response': [{'message': 'Je wil films en series streamen bij Videoland. Dat snap ik.'},
   {'message': 'Je kan een abonnement aanvragen via de Videoland website. Of in Mijn Odido. Dan heb je alles op 1 plek.'},
   {'message': 'Hoe wil je Videoland aanvragen?'}],
  'button_label': 'Aanvragen'}]

## Manage Summary Prompt

In [14]:
prompt = """
Vat deze chat {text} samen en beschrijf wat de (potentiële) klant van het telecombedrijf Odido wil. 
Focus alleen op de klantinput in "user_msg" en "button_label". Negeer begroetingen, afscheid, smalltalk en beledigingen. 
Maak een bondige samenvatting in het Nederlands van maximaal 100 tekens.
"""

## Generate Summary

In [15]:
summary = call_llm_service_summarize(conversation, prompt)
formatted_output = format_summary(summary)
print(formatted_output)
print()

# Count the output
char_count, word_count = count_output(formatted_output)
print(f"Characters: {char_count}")
print(f"Words: {word_count}")

Summary:
De klant wil informatie over het aanvragen van een Videoland-abonnement.

Characters: 81
Words: 11


## Update the prompt in Firestore Database (code uses the uploaded one)

In [16]:
# firestore_client = FirestoreClient()
# collection = firestore_client.collection("llm-prompts")
# document = collection.document("fulfillment-webhook")
# document.update({"summarize-chat-conversation": prompt})

# Manage Parameters and Call Que Prompt

In [17]:
service = """
Voorbeelden: 
- Vragen over facturen en betalingen (bijvoorbeeld uitleg over de factuur, incassodatum, betalingstermijn)
- Verzoeken voor afbetalingsregelingen
- Vragen over extra kosten of inflatiecorrectie
- Assistentie bij pin/puk-codes
- Hulp met de Mijn Odido omgeving (inclusief wachtwoordproblemen)
- Wijzigen van persoonlijke of betaalgegevens
- Verzoeken voor contractsovername
- Aanvragen van een vervangende simkaart
- Assistentie bij nummerportering
- Informatie over dienstgebruik en tarieven
- Aanpassen van extra's of bundels
"""

telesales = """
Voorbeelden:
- Afsluiten van een nieuw abonnement
- Verlengen van een bestaand abonnement
- Opzeggen van een abonnement (zowel tijdens als na de contractperiode)
"""

techniek = """
Voorbeelden:
- Problemen met thuisinternet (geen verbinding of slechte kwaliteit)
- Problemen met mobiel internet
- Slechte TV-ontvangst of niet-werkende TV-kastjes
- Router-gerelateerde problemen
- Problemen met telefonisch bereik
- Niet-beschikbare TV-zenders
- Algemene storingen met diensten
"""

activatie = """
Voorbeelden:
- Vragen over de planning van de monteur
- Informatie over de aansluitdatum
- Problemen na aansluiting (bijvoorbeeld wel aangesloten maar geen diensten)
- Melden van niet-verschenen monteurs
"""


In [78]:
customer_comment = """er is een storing"""

In [79]:
tag_prompt = """
Analyseer de contactreden van een (potentiële) klant van Odido, een telecombedrijf, en categoriseer deze in de juiste klantenservice queue.

Gegeven informatie:
- Primaire contactreden: {text}
- Aanvullende klantopmerking: {customer_comment}

Beschikbare queues:
1. Queue: Service
Omschrijving: Behandelt vragen over facturering, administratieve zaken en algemene informatie over diensten en accounts.
Voorbeelden: {service}
2. Queue: Telesales
Omschrijving: Handelt abonnementsaanvragen, nieuwe bestellingen, verlengingen en opzeggingen af.
Voorbeelden: {telesales}
3. Queue: Techniek
Omschrijving: Lost technische problemen op gerelateerd aan internet, TV en mobiele diensten.
Voorbeelden: {techniek}
4. Queue: Activatie
Omschrijving: Begeleidt nieuwe klanten bij het aansluitproces van thuisinternet en/of TV.
Voorbeelden: {activatie}

Instructies:
1. Analyseer de contactreden en eventuele aanvullende opmerking.
2. Analyseer de queues met de bijbehorende omschrijvingen en voorbeelden.
3. Geef een beknopte argumentatie (maximaal 150 tekens) waarom de klantvraag bij een bepaalde queue zou passen.
4. Kies op basis van deze argumentatie de meest geschikte queue.

Vereiste output:
Genereer een JSON-object met de volgende structuur:
{{
    "queue": "<naam van de gekozen queue>",
    "reason": "<beknopte reden voor de gekozen queue>"
}}

Zorg ervoor dat de output strikt voldoet aan dit JSON-formaat, zonder extra witruimte of aanvullende tekst.
"""

# Tag Call Queue

In [80]:
call_queue = call_llm_service_queue(summary, customer_comment, service, telesales, techniek, activatie, tag_prompt)
print(f"Tag prompt output: {call_queue}")

Summary: De klant wil informatie over het aanvragen van een Videoland-abonnement.
Customer Comment: er is een storing
Data prepared: {'custom': {'text': 'De klant wil informatie over het aanvragen van een Videoland-abonnement.', 'customer_comment': 'er is een storing', 'service': "\nVoorbeelden: \n- Vragen over facturen en betalingen (bijvoorbeeld uitleg over de factuur, incassodatum, betalingstermijn)\n- Verzoeken voor afbetalingsregelingen\n- Vragen over extra kosten of inflatiecorrectie\n- Assistentie bij pin/puk-codes\n- Hulp met de Mijn Odido omgeving (inclusief wachtwoordproblemen)\n- Wijzigen van persoonlijke of betaalgegevens\n- Verzoeken voor contractsovername\n- Aanvragen van een vervangende simkaart\n- Assistentie bij nummerportering\n- Informatie over dienstgebruik en tarieven\n- Aanpassen van extra's of bundels\n", 'telesales': '\nVoorbeelden:\n- Afsluiten van een nieuw abonnement\n- Verlengen van een bestaand abonnement\n- Opzeggen van een abonnement (zowel tijdens als na